In [ ]:
# Set sys path to root directory
import sys
sys.path.append("..")
from utilities.utils import evaluate_pii_detection, evaluate_non_pii_table, load_json_data, evaluate_pii_detection_multiclass, evaluate_pii_reflection_binary

# Path to PII file to evaluate
pii_path = "sensitive-data-detection/results/gittables.json"

# Load PII data
pii_data = load_json_data(pii_path)

# Assert error if pii_data is not dict


In [ ]:
# Path to PII file to evaluate
pii_path = "sensitive-data-detection/results/gittables.json"

# Load PII data
pii_data = load_json_data(pii_path)

# Assert error if pii_data is not dict
assert type(pii_data) == dict, "PII data is not a dict"

In [3]:
evaluate_pii_detection(pii_data, "Unsloth/gemma-2-9b-it", show_fps=True, show_fns=True)

/Users/liangtelkamp/miniconda3/envs/thesis/lib/python3.10/site-packages/numpy/lib/_function_base_impl.py:552: RuntimeWarning: Mean of empty slice.
  avg = a.mean(axis, **keepdims_kw)
/Users/liangtelkamp/miniconda3/envs/thesis/lib/python3.10/site-packages/numpy/_core/_methods.py:145: RuntimeWarning: invalid value encountered in scalar divide
  ret = ret.dtype.type(ret / rcount)


{'precision': 0.0, 'recall': 0.0, 'f1': 0.0, 'accuracy': nan}

## Evaluating PII Detection Models

In [4]:
# Multiclass PII detection evaluation
results_multiclass = evaluate_pii_detection_multiclass(
    data=pii_data,
    ground_truth_key="pii_detection",
    prediction_key="pii_detection_gpt-4o-mini",
    # show_confusion_matrix=False,
    show_misclassifications=True,
    show_report_table=True
)



### Classification Report
+-------------------------+-------------+----------+------------+-----------+
| Class                   | Precision   | Recall   | F1-Score   | Support   |
+=========================+=============+==========+============+===========+
| DATE_OF_BIRTH           | 0.571       | 0.941    | 0.711      | 17        |
+-------------------------+-------------+----------+------------+-----------+
| DISABILITY_GROUP        | 0.000       | 0.000    | 0.000      | 0         |
+-------------------------+-------------+----------+------------+-----------+
| EMAIL_ADDRESS           | 1.000       | 0.968    | 0.984      | 31        |
+-------------------------+-------------+----------+------------+-----------+
| ETHNIC_GROUP            | 0.917       | 0.611    | 0.733      | 18        |
+-------------------------+-------------+----------+------------+-----------+
| GENDER                  | 1.000       | 0.714    | 0.833      | 14        |
+-------------------------+----------

## Evaluating Reflection PII

In [5]:
evaluate_pii_reflection_binary(
    data=pii_data,
    ground_truth_key="pii_reflection",
    prediction_key="pii_detection_qwen3_8b_ft_reflection_unsloth/qwen3-14b",
    # show_fps=True,
    # show_fns=True,
    show_report_table=True,
)


### Binary Classification Report
+---------------+-------------+----------+------------+-----------+
| Class         | Precision   | Recall   | F1-Score   | Support   |
+===============+=============+==========+============+===========+
| NON_SENSITIVE | 0.985       | 0.946    | 0.965      | 1773      |
+---------------+-------------+----------+------------+-----------+
| SENSITIVE     | 0.734       | 0.910    | 0.812      | 288       |
+---------------+-------------+----------+------------+-----------+
|               |             |          |            |           |
+---------------+-------------+----------+------------+-----------+
| Overall       | 0.734       | 0.910    | 0.812      | 2061      |
+---------------+-------------+----------+------------+-----------+

### Class Distribution
+---------------+---------+--------------+
| Class         |   Count | Percentage   |
+===============+=========+==============+
| NON_SENSITIVE |    1773 | 86.0%        |
+---------------+-----

{'precision': 0.7338935574229691,
 'recall': 0.9097222222222222,
 'f1': 0.8124031007751938,
 'accuracy': 0.9412906356137797,
 'per_class': {'non_sensitive': {'precision': np.float64(0.9847417840375586),
   'recall': np.float64(0.9464184997179921),
   'f1': np.float64(0.9651998849582973),
   'support': np.int64(1773)},
  'sensitive': {'precision': np.float64(0.7338935574229691),
   'recall': np.float64(0.9097222222222222),
   'f1': np.float64(0.8124031007751938),
   'support': np.int64(288)}},
 'total_samples': 2061,
 'class_distribution': {'non_sensitive': 1773, 'sensitive': 288},
 'false_positives': 95,
 'false_negatives': 26}

## Code for overleaf table

In [6]:
# PII detection evaluation overleaf table 1
models = {
    "Google DLP": "dlp_custom",
    "Presidio": "presidio",
    "GPT-4o-mini": "gpt-4o-mini",
    "GPT-4o-mini no records": "gpt-4o-mini_no_records",
    "GPT-4o": "gpt-4o",
    "Gemma 2 9B": "unsloth/gemma-2-9b-it-bnb-4bit",
    "Gemma 2 9B FT": "/home/ltelkamp/detecting-sensitive-data/results/unsloth/gemma-2-9b-bnb-4bit-ft",
    "Gemma 3 12B": "unsloth/gemma-3-12b-it-bnb-4bit",
    "Qwen3 8B": "unsloth/qwen3-8b",
    "Qwen3 8B FT": "/home/ltelkamp/detecting-sensitive-data/results/unsloth/Qwen3-8B-ft",
    "Qwen3 14B": "unsloth/qwen3-14b",
    "Aya Expanse 8B": "CohereLabs/aya-expanse-8b"
}


for key, value in models.items():
    if key == "Presidio" or key == "Google DLP":
        prediction_key = value
    else:
        prediction_key = f"pii_detection_{value}"
    # Multiclass PII detection evaluation
    results_multiclass = evaluate_pii_detection_multiclass(
        data=pii_data,
        ground_truth_key="pii_detection",
        prediction_key=prediction_key,
        # show_report_table=False,
        show_confusion_matrix=False,
        show_misclassifications=False,
        show_report_table=False
    )
    scores = (results_multiclass['weighted_avg_pii_only'])
    scores_macro = (results_multiclass['macro_avg_pii_only'])
    prec, rec, f1 = scores['precision'], scores['recall'], scores['f1-score']
    prec_macro, rec_macro, f1_macro = scores_macro['precision'], scores_macro['recall'], scores_macro['f1-score']
    print(f"{key} weighted & {prec:.3f} & {rec:.3f} & {f1:.3f} \\\\")
    print(f"{key} macro & {prec_macro:.3f} & {rec_macro:.3f} & {f1_macro:.3f} \\\\")


Google DLP weighted & 0.713 & 0.324 & 0.298 \\
Google DLP macro & 0.409 & 0.322 & 0.296 \\
Presidio weighted & 0.198 & 0.245 & 0.208 \\
Presidio macro & 0.158 & 0.182 & 0.156 \\
GPT-4o-mini weighted & 0.578 & 0.366 & 0.422 \\
GPT-4o-mini macro & 0.735 & 0.503 & 0.555 \\
GPT-4o-mini no records weighted & 0.393 & 0.252 & 0.284 \\
GPT-4o-mini no records macro & 0.581 & 0.408 & 0.442 \\
GPT-4o weighted & 0.962 & 0.662 & 0.730 \\
GPT-4o macro & 0.702 & 0.664 & 0.668 \\
Gemma 2 9B weighted & 0.960 & 0.537 & 0.605 \\
Gemma 2 9B macro & 0.763 & 0.608 & 0.623 \\
Gemma 2 9B FT weighted & 0.947 & 0.775 & 0.833 \\
Gemma 2 9B FT macro & 0.748 & 0.687 & 0.703 \\
Gemma 3 12B weighted & 0.861 & 0.812 & 0.819 \\
Gemma 3 12B macro & 0.690 & 0.722 & 0.681 \\
Qwen3 8B weighted & 0.919 & 0.548 & 0.581 \\
Qwen3 8B macro & 0.723 & 0.618 & 0.627 \\
Qwen3 8B FT weighted & 0.968 & 0.932 & 0.941 \\
Qwen3 8B FT macro & 0.861 & 0.807 & 0.819 \\
Qwen3 14B weighted & 0.933 & 0.819 & 0.858 \\
Qwen3 14B macro & 0.720 

In [7]:
# Every pII type as sensitive
from utilities.utils import evaluate_pii_as_reflection

results = evaluate_pii_as_reflection(
    data=pii_data,
    ground_truth_key="pii_reflection",
    prediction_key=f"pii_detection",
)

print(f"& ground truth & {results['precision']:.3f} & {results['recall']:.3f} & {results['f1']:.3f} \\\\")

models = {
    "Google DLP": "dlp_custom",
    "Presidio": "presidio",
    "GPT-4o-mini": "gpt-4o-mini",
    # "GPT-4o": "gpt-4o",
    "Gemma 2 9B": "unsloth/gemma-2-9b-it-bnb-4bit",
    # "Gemma 2 9B FT": "/home/ltelkamp/detecting-sensitive-data/results/unsloth/gemma-2-9b-bnb-4bit-ft",
    "Gemma 3 12B": "unsloth/gemma-3-12b-it-bnb-4bit",
    "Qwen3 8B": "unsloth/qwen3-8b",
    # "Qwen3 8B FT": "/home/ltelkamp/detecting-sensitive-data/results/unsloth/Qwen3-8B-ft",
    "Qwen3 14B": "unsloth/qwen3-14b",
    "Aya Expanse 8B": "CohereLabs/aya-expanse-8b"
}

for key, value in models.items():
    if key == "Presidio" or key == "Google DLP":
        prediction_key = value
    else:
        prediction_key = f"pii_detection_{value}"
    results = evaluate_pii_as_reflection(
        data=pii_data,
        ground_truth_key="pii_reflection",
        prediction_key=prediction_key,
    )

    print(f"{key} & {results['precision']:.3f} & {results['recall']:.3f} & {results['f1']:.3f} \\\\")

& ground truth & 0.527 & 1.000 & 0.690 \\
Google DLP & 0.195 & 0.628 & 0.297 \\
Presidio & 0.191 & 0.618 & 0.292 \\
GPT-4o-mini & 0.856 & 0.639 & 0.732 \\
Gemma 2 9B & 0.740 & 0.819 & 0.778 \\
Gemma 3 12B & 0.487 & 0.941 & 0.642 \\
Qwen3 8B & 0.742 & 0.868 & 0.800 \\
Qwen3 14B & 0.565 & 0.972 & 0.714 \\
Aya Expanse 8B & 0.812 & 0.674 & 0.736 \\


In [8]:
# PII groundtruth reflection evaluation
models = {
    # "Presidio": "presidio",
    "GPT-4o-mini": "gpt-4o-mini",
    # "GPT-4o": "gpt-4o",
    "Gemma 2 9B": "unsloth/gemma-2-9b-it-bnb-4bit",
    # "Gemma 2 9B FT": "/home/ltelkamp/detecting-sensitive-data/results/unsloth/gemma-2-9b-bnb-4bit-ft",
    "Gemma 3 12B": "unsloth/gemma-3-12b-it-bnb-4bit",
    "Qwen3 8B": "unsloth/qwen3-8b",
    # "Qwen3 8B FT": "/home/ltelkamp/detecting-sensitive-data/results/unsloth/Qwen3-8B-ft",
    "Qwen3 14B": "unsloth/qwen3-14b",
    "Aya Expanse 8B": "CohereLabs/aya-expanse-8b",
}

for key, value in models.items():
    results = evaluate_pii_reflection_binary(
        data=pii_data,
        ground_truth_key="pii_reflection",
        prediction_key=f"pii_detection_gt_reflection_{value}",
        # show_fps=True,
        # show_fns=True,
        show_report_table=False,
    )
    prec, rec, f1 = results['precision'], results['recall'], results['f1']
    print(f"{key} & {prec:.3f} & {rec:.3f} & {f1:.3f} \\\\")

key="Qwen3 8B FT & GPT-4o-mini"
results = evaluate_pii_reflection_binary(
    data=pii_data,
    ground_truth_key="pii_reflection",
    prediction_key=f"pii_detection_qwen3_8B_ft_reflection_gpt_4o_mini",
    # show_fps=True,
    # show_fns=True,
    show_report_table=False,
)
prec, rec, f1 = results['precision'], results['recall'], results['f1']
print(f"{key} & {prec:.3f} & {rec:.3f} & {f1:.3f} \\\\")

GPT-4o-mini & 0.932 & 0.944 & 0.938 \\
Gemma 2 9B & 0.705 & 0.913 & 0.796 \\
Gemma 3 12B & 0.883 & 0.816 & 0.848 \\
Qwen3 8B & 0.544 & 0.997 & 0.703 \\
Qwen3 14B & 0.683 & 0.951 & 0.795 \\
Aya Expanse 8B & 0.552 & 0.997 & 0.710 \\
Qwen3 8B FT & GPT-4o-mini & 0.902 & 0.861 & 0.881 \\


In [ ]:
# import json
# with open("sensitive-data-detection/results/gittables.json", "r") as f:
#     data = json.load(f)

# with open("sensitive-data-detection/results/gittables_presidio.json", "r") as f:
#     presidio_data = json.load(f)

# # print(data)
# for table, value in data.items():
#     for column_name, column_value in value['columns'].items():
#         # print(f"Table: {table}")
#         # print(f"Column name: {column_name}")
#         # print(presidio_data[table]['columns'])
#         # print(presidio_data[table]['columns'][column_name])
#         # column_value['presidio'] = presidio_data[table]['columns'][column_name]
#         # If all records are None values, then
#         if all(record is None for record in column_value['records']):
#             column_value['pii_reflection_unsloth/qwen3-14b'] = 'NON_SENSITIVE'
#             column_value['pii_reflection_unsloth/qwen3-8b'] = 'NON_SENSITIVE'
#             column_value['pii_reflection_unsloth/gemma-3-12b-it-bnb-4bit'] = 'NON_SENSITIVE'
#             column_value['pii_reflection_CohereLabs/aya-expanse-8b'] = 'NON_SENSITIVE'
#             column_value['pii_reflection_gpt-4o-mini'] = 'NON_SENSITIVE'
#             column_value['pii_reflection_gpt-4o'] = 'NON_SENSITIVE'
#             column_value['presidio'] = 'NON_SENSITIVE'
#             column_value['dlp_custom'] = 'NON_SENSITIVE'
#         # try:a
#         #     del column_value['pii_detection_gpt-4o-mini_no_records']
#         # except:
#         #     continue

# # Save the data to a json file
# # with open("sensitive-data-detection/results/gittables_modified.json", "w") as f:
# #     json.dump(data, f, indent=4)

In [ ]:
# # Save the data to a json file

# # Save the data to a json file
# with open("sensitive-data-detection/results/gittables.json", "w") as f:
#     json.dump(data, f, indent=4)

In [ ]:
import json
with open("sensitive-data-detection/results/gittables.json", "r") as f:
    data = json.load(f)

results = {}
# print(data)
for table, value in data.items():
    for column_name, column_value in value['columns'].items():
        pii_type = column_value['pii_detection']
        if pii_type not in results:
            results[pii_type] = {"total": 0, "sensitive": 0}
        results[pii_type]["total"] += 1
        if column_value['pii_reflection'] != "NON_SENSITIVE":
            results[pii_type]["sensitive"] += 1

print(results)

for pii_type, value in results.items():
    print(f"{pii_type}: {value['sensitive']}/{value['total']}")

{'GENERIC_IDENTIFIER': {'total': 216, 'sensitive': 26}, 'PERSON_NAME': {'total': 93, 'sensitive': 92}, 'None': {'total': 1514, 'sensitive': 0}, 'GENDER': {'total': 14, 'sensitive': 11}, 'MARITAL_STATUS': {'total': 1, 'sensitive': 1}, 'PHONE_NUMBER': {'total': 21, 'sensitive': 20}, 'EMAIL_ADDRESS': {'total': 31, 'sensitive': 30}, 'DATE_OF_BIRTH': {'total': 17, 'sensitive': 17}, 'URL': {'total': 11, 'sensitive': 1}, 'GEO_COORDINATES': {'total': 24, 'sensitive': 7}, 'ORGANIZATION_NAME': {'total': 27, 'sensitive': 1}, 'ZIPCODE': {'total': 27, 'sensitive': 26}, 'STREET_ADDRESS': {'total': 38, 'sensitive': 35}, 'ETHNIC_GROUP': {'total': 18, 'sensitive': 17}, 'IP_ADDRESS': {'total': 3, 'sensitive': 0}, 'OCCUPATION': {'total': 5, 'sensitive': 3}, 'SPOKEN_LANGUAGE': {'total': 1, 'sensitive': 1}}
GENERIC_IDENTIFIER: 26/216
PERSON_NAME: 92/93
None: 0/1514
GENDER: 11/14
MARITAL_STATUS: 1/1
PHONE_NUMBER: 20/21
EMAIL_ADDRESS: 30/31
DATE_OF_BIRTH: 17/17
URL: 1/11
GEO_COORDINATES: 7/24
ORGANIZATION_NA